In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!apt-get install git
!pip install accelerate -U
!git clone https://github.com/ryanzhumich/AESLC.git
import os
# Path to the 'enron_subject_line' folder within 'AESLC'
folder_path = '/content/AESLC/enron_subject_line'
# List contents of the 'enron_subject_line' folder
contents = os.listdir(folder_path)
print(contents)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (48/48), done.
['dev', 'test', 'train']


In [ ]:
import os
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"  #"gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def process_subject_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Customize this part based on your .subject file structure
        subject_text = file.read().strip()
        return {'File': os.path.basename(file_path), 'Subject': subject_text}

def create_csv(directory_path, output_csv):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.subject'):
            file_path = os.path.join(directory_path, filename)
            data.append(process_subject_file(file_path))

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)

input_directory = "/content/AESLC/enron_subject_line/train"
output_csv_file = "output.csv"
create_csv(input_directory, output_csv_file)

In [ ]:

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/output.csv')

# Assuming your column is named 'text_and_subject'
df['Text'] = df['Subject'].apply(lambda x: x.split('@subject')[0].strip())
df['Sub'] = df['Subject'].apply(lambda x: x.split('@subject')[1].strip() if '@subject' in x else '')

# Drop the original column if needed
df.drop(columns=['Subject','File'], inplace=True)



In [ ]:
df.head(5)

,Text,Sub
0,"Darron, Here's the detail we discussed.\nLet...",NSS Detail
1,"Phil, I have attached a presentation that I th...",Rockies Presentation
2,There is a P&L Summary for east desk in the Ne...,P&L summ
3,Are you still working every other day?\nPlease...,SITA
4,"Dorie, I have to question the effectiveness of...",2001 ENA/EGM/EIM CUSTOMER SKI PROGRAM


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df.iterrows():
    input_text = row['Text']
    subject = row['Sub']
    input_text = " ".join(input_text.split()[:300])
    out = input_text + " \n@subject\n " + subject + " \n<EMAIL_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/gpt_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

# Train the model
trainer.train()


Step,Training Loss
500,3.382600
1000,3.241200
1500,3.160400
2000,3.125800
2500,3.097800
3000,3.039500
3500,3.028700
4000,2.891300
4500,2.818700
5000,2.832500


TrainOutput(global_step=10827, training_loss=2.8782711731954835, metrics={'train_runtime': 3677.9151, 'train_samples_per_second': 11.775, 'train_steps_per_second': 2.944, 'total_flos': 5979531382272000.0, 'train_loss': 2.8782711731954835, 'epoch': 3.0})

In [ ]:
model_output_path = "/content/gpt_model"

In [ ]:
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/gpt_model/tokenizer_config.json',
 '/content/gpt_model/special_tokens_map.json',
 '/content/gpt_model/vocab.json',
 '/content/gpt_model/merges.txt',
 '/content/gpt_model/added_tokens.json')

In [ ]:
def generate_response(model, tokenizer, prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Load the fine-tuned model and tokenizer
my_model = GPT2LMHeadModel.from_pretrained(model_output_path)
my_tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [ ]:
my_model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "Dan,  Ref. is made to yr e-mail today 30th re ETA Oman.Have just received following from the Superintendent, Nils Reite: ++ Just to give you a brief update on the progressing repair work at the sunny Algeciras roads.- External assistance team:   * One service engineer from Turbo Technik for the manoeuvre valve   * Two technicians from NorPol Marine Service to assist above   * Three pipe fitters from NorPol Marine Services to assist the Ch.eng.- All repair work progressing nicely.As for the ahead steam control valve we   have found an external linkage shaft to be the reason for the problem.This   is now being extracted and will be polished and refitted.- We are aiming for completion on 2001-05-01 late evening and sailing on   2001-05-02 early morning.brgds/NBR ++ This was good news and it seems that we can follow our schedule Will revert tomorrow with an update. Best regards"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Generated response: Dan,  Ref. is made to yr e-mail today 30th re ETA Oman.Have just received following from the Superintendent, Nils Reite: ++ Just to give you a brief update on the progressing repair work at the sunny Algeciras roads.- External assistance team:   * One service engineer from Turbo Technik for the manoeuvre valve   * Two technicians from NorPol Marine Service to assist above   * Three pipe fitters from NorPol Marine Services to assist the Ch.eng.- All repair work progressing nicely.As for the ahead steam control valve we   have found an external linkage shaft to be the reason for the problem.This   is now being extracted and will be polished and refitted.- We are aiming for completion on 2001-05-01 late evening and sailing on   2001-05-02 early morning.brgds/NBR ++ This was good news and it seems that we can follow our schedule Will revert tomorrow with an update. Best regards,                                                                                             

In [ ]:
import os
import csv
from tqdm import tqdm  # Import tqdm for progress bar


# Folder path containing the files
folder_path = "/content/AESLC/enron_subject_line/dev"

# Output CSV file path
csv_output_path = "model_dev_output.csv"

# Model saving path
model_save_path = "/content/model"
os.makedirs(model_save_path, exist_ok=True)

# Initialize an empty list to store data
data = []

# Number of iterations to save the model
save_model_interval = 5

# Iterate through files in the folder
for iteration, filename in enumerate(tqdm(os.listdir(folder_path), desc="Processing files")):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r') as file:
          try:
            content = file.read()

            # Extracting prompt and annotations using string splitting
            prompt = content.split('@subject')[0].strip()[:800] + " @subject" if '@subject' in content else ''
            print(f"\nIteration: {iteration}")
            # print(f"Modified Prompt: {prompt}")

            subject = content.split('@subject')[1].split('@')[0].strip() if '@subject' in content else ''

            ann0 = content.split('@ann0')[1].split('@')[0].strip() if '@ann0' in content else ''
            # print(f"Annotation 0: {ann0}")

            ann1 = content.split('@ann1')[1].split('@')[0].strip() if '@ann1' in content else ''
            # print(f"Annotation 1: {ann1}")

            ann2 = content.split('@ann2')[1].split('@')[0].strip() if '@ann2' in content else ''
            # print(f"Annotation 2: {ann2}")

            # Generating a response (replace with your actual model inference logic)
            response = generate_response(my_model, my_tokenizer, prompt)
            # print(f"Generated Response: {response}")

            # Append data to the list
            data.append([prompt, subject, ann0, ann1, ann2, response])
          except Exception as e:
                print(f"Error reading file {filename}: {e}")

    # Save the model every 5 iterations
    # if iteration > 0 and iteration % save_model_interval == 0:
    #     model_save_filename = f"model_iteration_{iteration}.pth"
    #     model_save_filepath = os.path.join(model_save_path, model_save_filename)
    #     my_model.save_pretrained(model_save_filepath)
    #     print(f"Model saved at: {model_save_filepath}")

# Write data to CSV file
with open(csv_output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

    # Write data rows
    csvwriter.writerows(data)

print(f"CSV file '{csv_output_path}' created successfully.")


Processing files:   0%|          | 0/1960 [00:00<?, ?it/s]


Iteration: 0


Processing files:   0%|          | 1/1960 [00:04<2:42:17,  4.97s/it]


Iteration: 1


Processing files:   0%|          | 2/1960 [00:07<1:59:00,  3.65s/it]

...


Iteration: 1958


Processing files: 100%|█████████▉| 1959/1960 [38:41<00:01,  1.21s/it]


Iteration: 1959


Processing files: 100%|██████████| 1960/1960 [38:42<00:00,  1.18s/it]

CSV file 'model_dev_output.csv' created successfully.


In [ ]:
import os
import csv
from tqdm import tqdm  # Import tqdm for progress bar


# Folder path containing the files
folder_path = "/content/AESLC/enron_subject_line/test"

# Output CSV file path
csv_output_path = "model_test_output.csv"

# Model saving path
model_save_path = "/content/model"
os.makedirs(model_save_path, exist_ok=True)

# Initialize an empty list to store data
data = []

# Number of iterations to save the model
save_model_interval = 5

# Iterate through files in the folder
for iteration, filename in enumerate(tqdm(os.listdir(folder_path), desc="Processing files")):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r') as file:
          try:
            content = file.read()

            # Extracting prompt and annotations using string splitting
            prompt = content.split('@subject')[0].strip()[:800] + " @subject" if '@subject' in content else ''
            print(f"\nIteration: {iteration}")
            # print(f"Modified Prompt: {prompt}")

            subject = content.split('@subject')[1].split('@')[0].strip() if '@subject' in content else ''

            ann0 = content.split('@ann0')[1].split('@')[0].strip() if '@ann0' in content else ''
            # print(f"Annotation 0: {ann0}")

            ann1 = content.split('@ann1')[1].split('@')[0].strip() if '@ann1' in content else ''
            # print(f"Annotation 1: {ann1}")

            ann2 = content.split('@ann2')[1].split('@')[0].strip() if '@ann2' in content else ''
            # print(f"Annotation 2: {ann2}")

            # Generating a response (replace with your actual model inference logic)
            response = generate_response(my_model, my_tokenizer, prompt)
            # print(f"Generated Response: {response}")

            # Append data to the list
            data.append([prompt, subject, ann0, ann1, ann2, response])
          except Exception as e:
                print(f"Error reading file {filename}: {e}")

    # Save the model every 5 iterations
    # if iteration > 0 and iteration % save_model_interval == 0:
    #     model_save_filename = f"model_iteration_{iteration}.pth"
    #     model_save_filepath = os.path.join(model_save_path, model_save_filename)
    #     my_model.save_pretrained(model_save_filepath)
    #     print(f"Model saved at: {model_save_filepath}")

# Write data to CSV file
with open(csv_output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

    # Write data rows
    csvwriter.writerows(data)

print(f"CSV file '{csv_output_path}' created successfully.")


Processing files:   0%|          | 0/1906 [00:00<?, ?it/s]


Iteration: 0


Processing files:   0%|          | 1/1906 [00:01<34:51,  1.10s/it]


Iteration: 1


Processing files:   0%|          | 2/1906 [00:02<35:28,  1.12s/it]

...

Processing files:  87%|████████▋ | 1660/1906 [33:05<04:41,  1.14s/it]


Iteration: 1660


Processing files:  87%|████████▋ | 1661/1906 [33:06<04:39,  1.14s/it]


Iteration: 1661


Processing files:  87%|████████▋ | 1662/1906 [33:07<04:47,  1.18s/it]

In [ ]:
my_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
data_dev = pd.read_csv("/content/model_dev_output.csv")

In [ ]:
data_test = pd.read_csv("/content/model_test_output.csv")

In [ ]:
data_dev.head(5)

,Prompt,Subject,Annotation 0,Annotation 1,Annotation 2,Generated Subject
0,"Part II: FERC Writing Handbook 1.\n""WHEREFOR...","so ya wanna be a FERC lawyer, Part II",part ii: ferc writing handbook i-- pay close...,ferc writing handbook,grammar and format changes for all pleadings,"Part II: FERC Writing Handbook 1.\n""WHEREFOR..."
1,"Hi Phillip, This message is to confirm our me...",Confirmation: Risk Management Simulation Meeti...,"meeting confimation - tues, 8/10",meeting confirmation,meeting confirmed,"Hi Phillip, This message is to confirm our me..."
2,Rick Buy has requested that we maintain an up-...,Emergency Contact Info,employee contat list,please verify your contact information,update employee info,Rick Buy has requested that we maintain an up-...
3,I'm on the phone with GE.\nTheir initial respo...,Blue Dog,ge status update,letter to ge re ownership of llc and payment,ge thoughts and wants,I'm on the phone with GE.\nTheir initial respo...
4,Please remember that MELISSA MURPHY (X31886) h...,Power confirmations,financial power confirmation notice,contact only melissa murphy for financial powe...,handling of financila power confirmations and ...,Please remember that MELISSA MURPHY (X31886) h...


In [ ]:
def extract_subject(subject_text):
    start = subject_text.find('@subject') + len('@subject')
    end = subject_text.find('@subject', start)
    if end != -1:
        return subject_text[start:end].strip()
    else:
        return subject_text[start:].strip()

In [ ]:
data_dev['Generated Subject'] = data_dev['Generated Subject'].apply(extract_subject)

In [ ]:
data_test['Generated Subject'] = data_test['Generated Subject'].apply(extract_subject)

In [ ]:
data_dev.head(10)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9a26886a8153c96f3a785cade1169d9a490e005b022bd3e70ab2084d0b6421f6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from evaluate import load
from collections import defaultdict
from tqdm import tqdm

def metric(data):
    sub,ann0,ann1,ann2,gen_subject  = ["Subject","Annotation 0","Annotation 1","Annotation 2","Generated Subject"]
    rouge = load("rouge")
    meteor = load("meteor")
    score = defaultdict(int)

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_s = rouge.compute(predictions=[x[gen_subject]],references=[x[sub]])
        rouge_h = rouge.compute(predictions=[x[gen_subject]] ,references=[[x[ann0],x[ann1],x[ann2]]])

        score["rouge_1_sub"] += rouge_s['rouge1']
        score["rouge_2_sub"] += rouge_s['rouge2']
        score["rouge_L_sub"] += rouge_s['rougeL']

        score["rouge_1_human"] += rouge_h['rouge1']
        score["rouge_2_human"] += rouge_h['rouge2']
        score["rouge_L_human"] += rouge_h['rougeL']


        meteor_s = meteor.compute(predictions=[x[gen_subject]],references=[x[sub]])
        meteor_h = meteor.compute(predictions=[x[gen_subject]] ,references=[[x[ann0],x[ann1],x[ann2]]])

        score["meteor_sub"] += meteor_s["meteor"]
        score["meteor_human"] += meteor_h["meteor"]

    print("The evalution scores are as follows:")

    for name,val in score.items():
        print(f"{name}: {val/len(data)}")

In [ ]:
metric(data_dev)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
100%|██████████| 1949/1949 [13:18<00:00,  2.44it/s]

The evalution scores are as follows:
rouge_1_sub: 0.1345339458927429
rouge_2_sub: 0.05838364988467858
rouge_L_sub: 0.12667269977974854
rouge_1_human: 0.24414782681311117
rouge_2_human: 0.13907358844951098
rouge_L_human: 0.22959092157827013
meteor_sub: 0.18024229783743606
meteor_human: 0.3204199127393829


In [ ]:
# @title
metric(data_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 1901/1901 [13:08<00:00,  2.41it/s]

The evalution scores are as follows:
rouge_1_sub: 0.13453935268453035
rouge_2_sub: 0.06215518077308029
rouge_L_sub: 0.1276900913113724
rouge_1_human: 0.2503761782900848
rouge_2_human: 0.142701078236341
rouge_L_human: 0.23369781906585105
meteor_sub: 0.17368387106228747
meteor_human: 0.32691471270697
